In [1]:
import pandas as pd
import sys
from data_reader import fes_reader, perf_reader
from tqdm import tqdm
import bisect
import json
import datetime

In [2]:
# конвертация путей файлов в зависимости от системы
def replace_slash(file_path):
    platform = sys.platform
    slash_map = {'win32': '\\',
                'cygwin': '\\',
                'darwin': '/',
                'linux2': '/'}
    if platform not in slash_map.keys(): platform = 'linux2'
    return file_path.replace('\\', slash_map[platform])

In [3]:
perf_path1 = replace_slash('input_data\\' + 'armitz/miner/ARX_PERF.xlsx')
fes_path1 = replace_slash('input_data\\armitz/miner/QINF.xlsx')

perf_path = replace_slash('input_data\\' + 'Перфорация.json')
fes_path = replace_slash('input_data\\РИГИС.json')

In [6]:
with open(fes_path, 'r') as f:
    data = json.dumps(json.load(f))
with open(perf_path, 'r') as f:
    data_perf = json.dumps(json.load(f))

In [7]:
perf_df = pd.read_json(data_perf, orient='records')

In [5]:
fes_df = pd.read_json(data, orient='records')

In [8]:
perf_df1 = pd.read_excel(perf_path1, engine='openpyxl', skiprows=1)
fes_df1 = pd.read_excel(fes_path1, engine='openpyxl', skiprows=1)

In [9]:
fes_df

,skw_id,skw,ngdu,cex,krow,podosh,plast,plast_name,sost_pl,kollek,...,vod_m,poris,pr_abs,pr_faz,glin,nnas,nas_pl,litolog,krow_abs,pod_abs
0,2382,20375,20,8,1807.2,1809.6,8219,Д0+Д1Г2+3,0,0,...,0.0,22.7,629.5,208.2,2.1,NaN,неясный хар.насыщ.,заглин.песчаник,0.0,0.0
1,2384,20382,20,2,1932.8,1935.0,8219,Д0+Д1Г2+3,2,0,...,0.0,20.5,501.2,155.9,1.6,82.5,водяная зона,песчаник,0.0,0.0
2,955,20612,20,2,1790.4,1791.6,8219,Д0+Д1Г2+3,0,0,...,0.0,13.8,57.2,23.2,3.4,59.2,неясный хар.насыщ.,алевролит,0.0,0.0
3,955,20612,20,2,1792.0,1793.6,8219,Д0+Д1Г2+3,1,0,...,0.0,17.0,265.1,80.0,0.0,73.8,неясный хар.насыщ.,заглин.песчаник,0.0,0.0
4,955,20612,20,2,1800.2,1801.4,8219,Д0+Д1Г2+3,0,0,...,0.0,22.1,757.2,224.5,0.4,0.0,неясный хар.насыщ.,песчаник,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69947,4003,14989,20,5,1627.6,1628.4,1874,Д0+Д1A,1,25305,...,0.0,NaN,NaN,NaN,NaN,NaN,слабонефтенасыщ.пл.,коллектор глинистый,0.0,0.0
69948,589,3102Д,20,2,1704.0,1704.8,8207,Д0+Д1Б3,0,25305,...,0.0,19.0,151.0,82.8,81.0,60.3,слабонефтенасыщ.пл.,коллектор глинистый,0.0,0.0
69949,4002,14988,20,5,1641.0,1642.0,8211,Д0+Д1B,1,25304,...,0.0,15.2,84.4,33.0,2.7,69.0,нефтенасыщенный пл.,уплотненный коллектор,0.0,0.0
69950,3974,10809,20,5,1763.6,1765.2,1919,Д0,0,25305,...,0.0,15.0,34.0,28.0,NaN,59.5,слабонефтенасыщ.пл.,коллектор глинистый,0.0,0.0


In [10]:
fes_df1

,Цех,Скваж,Ствол,Зона,Профиль,Дата_бур,Мест,Площ,Гориз,Тек_заб,...,Н_нас,Характер_насыщения,Литология,Абс_кр,Абс_под,Мах_зен_уг,Глуб_мах_уг,Мin_зен_уг,D_эк,Н_цем
0,8,1,1.0,NaN,HНС,1950-02-22,1.0,905.0,531.0,NaN,...,NaN,NaN,не коллек,NaN,NaN,4.30,1000.0,0.1,168.0,NaN
1,8,1,1.0,NaN,HНС,NaT,1.0,905.0,531.0,NaN,...,69.5,NaN,алевролит,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,1,1.0,NaN,HНС,NaT,1.0,905.0,531.0,NaN,...,NaN,NaN,не коллек,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,1,1.0,NaN,HНС,NaT,1.0,905.0,531.0,NaN,...,NaN,NaN,не коллек,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,1,1.0,NaN,HНС,NaT,1.0,905.0,531.0,NaN,...,NaN,NaN,не коллек,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156269,9,32960,1.0,1.0,ОРЭ,2010-03-26,NaN,NaN,NaN,1926.0,...,52.9,нефть слабо,загл.песч.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156270,9,32960,1.0,1.0,ОРЭ,NaT,NaN,NaN,NaN,NaN,...,NaN,коллектор обводненны,песчаник,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156271,9,32960,1.0,NaN,ОРЭ,2010-03-26,1.0,908.0,531.0,1926.0,...,NaN,верхний известняк,не коллек,1478.6,1480.0,32.96,1300.0,7.8,146.0,68.0
156272,9,32960,1.0,2.0,ОРЭ,2010-03-26,NaN,NaN,NaN,1926.0,...,NaN,коллектор обводненны,загл.песч.,1482.1,1483.5,NaN,NaN,NaN,NaN,NaN


In [4]:
perf_ints = perf_reader(perf_path)
fes_dict = fes_reader(fes_path)
SOIL_CUT = 60

started reading perf xl
done reading perf xl
started reading fes xl
done reading fes xl
